In [1]:
import util as ut
import constantes as cons
from backtesting import Backtest, Strategy
import numpy as np
symbol =  str.upper('cfx'+'USDT')

data = ut.estrategia(symbol)

## BACKTESTING
class Predictor2(Strategy):
    def init(self):
        pass
   
    def next(self):
        if not self.position:
            if self.data.signal[-1] ==1:
                self.buy(size=1000, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
            elif self.data.signal[-1] ==-1:
                self.sell(size=1000, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
        else:
            pass

bt = Backtest(data, Predictor2, cash=1000, commission=.002, exclusive_orders=True)

output = bt.run()
bt.plot()
output



C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatte

Start                     2023-06-04 08:00:00
End                       2023-06-08 11:30:00
Duration                      4 days 03:30:00
Exposure Time [%]                        89.0
Equity Final [$]                  1017.786935
Equity Peak [$]                   1026.086935
Return [%]                           1.778694
Buy & Hold Return [%]              -15.797208
Return (Ann.) [%]                   86.786869
Volatility (Ann.) [%]               38.286907
Sharpe Ratio                         2.266751
Sortino Ratio                        7.526714
Calmar Ratio                        33.227938
Max. Drawdown [%]                   -2.611864
Avg. Drawdown [%]                   -0.445167
Max. Drawdown Duration        2 days 14:30:00
Avg. Drawdown Duration        0 days 09:14:00
# Trades                                   12
Win Rate [%]                        91.666667
Best Trade [%]                       2.333982
Worst Trade [%]                     -3.173974
Avg. Trade [%]                    

In [2]:
data

,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,deriv,signal,take_profit,stop_loss
2023-06-04 08:00:00,0.2701,0.2757,0.2698,0.2722,3790833.0,0.269819,0.273133,0.285006,0.002142,0.397436,0,0.000000,0.285006
2023-06-04 08:30:00,0.2722,0.2729,0.2716,0.2726,1242349.0,0.270084,0.273112,0.284883,0.002082,0.397436,0,0.000000,0.284883
2023-06-04 09:00:00,0.2726,0.2751,0.2725,0.2740,1776307.0,0.270457,0.273146,0.284775,0.002119,0.397436,0,0.000000,0.284775
2023-06-04 09:30:00,0.2741,0.2758,0.2737,0.2740,1817456.0,0.270794,0.273180,0.284667,0.002117,0.332309,0,0.000000,0.284667
2023-06-04 10:00:00,0.2741,0.2746,0.2732,0.2739,736627.0,0.271090,0.273208,0.284560,0.002066,0.267651,-1,0.271834,0.284560
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-08 09:30:00,0.2282,0.2287,0.2274,0.2279,1072594.0,0.227714,0.230337,0.243569,0.002358,0.319540,0,0.000000,0.243569
2023-06-08 10:00:00,0.2278,0.2285,0.2271,0.2279,1169973.0,0.227732,0.230242,0.243413,0.002290,0.332537,0,0.000000,0.243413
2023-06-08 10:30:00,0.2280,0.2305,0.2278,0.2299,1579013.0,0.227939,0.230228,0.243279,0.002319,0.380796,0,0.000000,0.243279
2023-06-08 11:00:00,0.2299,0.2306,0.2288,0.2291,1528713.0,0.228049,0.230184,0.243138,0.002282,0.336765,0,0.000000,0.243138
